# Use Foursquare to Examine Toronto Neighborhoods

In [1]:
# imports
import pandas as pd
import numpy as np
import requests

In [2]:
# read in csv created in the previous notebook
toronto_final = pd.read_csv('toronto_final.csv')
toronto_final.head()

,postalcode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
# CLIENT_ID = '' # your Foursquare ID
# CLIENT_SECRET = '' # your Foursquare Secret
# VERSION = '' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [4]:
# function to get nearby venues for each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['borough', 
                  'borough Latitude', 
                  'borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
# # get nearby venues for each postal code in Toronto
# toronto_venues = getNearbyVenues(names=toronto_final['borough'],
#                                    latitudes=toronto_final['Latitude'],
#                                    longitudes=toronto_final['Longitude']
#                                   )

In [6]:
# check new dataframe
toronto_venues.head()

,borough,borough Latitude,borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [7]:
# get venue counts by borough
toronto_venues.groupby(by='borough').count()

,borough Latitude,borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
borough,,,,,,
Central Toronto,112,112,112,112,112,112
Downtown Toronto,1296,1296,1296,1296,1296,1296
East Toronto,121,121,121,121,121,121
East York,78,78,78,78,78,78
Etobicoke,73,73,73,73,73,73
Mississauga,8,8,8,8,8,8
North York,244,244,244,244,244,244
Queen's Park,41,41,41,41,41,41
Scarborough,89,89,89,89,89,89


In [8]:
# calculate number of unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


In [9]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['borough'] = toronto_venues['borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# check shape 
toronto_onehot.shape

(2266, 272)

In [11]:
# group by borough
toronto_grouped = toronto_onehot.groupby('borough').mean().reset_index()
toronto_grouped

,borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,...,0.000000,0.008929,0.000000,0.000000,0.008929,0.000000,0.000000,0.000000,0.000000,0.008929
1,Downtown Toronto,0.000000,0.000772,0.000772,0.000772,0.000772,0.001543,0.002315,0.001543,0.015432,...,0.002315,0.011574,0.002315,0.000000,0.005401,0.000000,0.006944,0.000772,0.000772,0.002315
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024793,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016529
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.012821,0.000000,0.012821,0.000000,0.000000,0.000000,0.012821
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.008197,0.000000,0.004098,0.000000,0.000000,0.000000,0.000000,0.000000,0.008197,...,0.000000,0.000000,0.004098,0.004098,0.008197,0.000000,0.000000,0.004098,0.008197,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.024390
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011236,...,0.000000,0.000000,0.000000,0.000000,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010870,0.000000,0.000000,0.010870,0.000000,0.005435,0.000000,0.000000,0.010870


In [12]:
# print out the top 5 venues for each postal code
num_top_venues = 5

for hood in toronto_grouped['borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.06
2            Café  0.05
3            Park  0.05
4    Dessert Shop  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.05
2                Hotel  0.03
3           Restaurant  0.03
4  Japanese Restaurant  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.05
3                Café  0.04
4      Ice Cream Shop  0.04


----East York----
          venue  freq
0   Coffee Shop  0.06
1  Burger Joint  0.05
2          Park  0.05
3          Bank  0.04
4   Pizza Place  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1  Sandwich Place  0.07
2     Coffee Shop  0.05
3        Pharmacy  0.05
4            Park  0.04


----Mississauga----
                 venue  freq
0          Coffee Shop  0.25
1                Hotel  0.25
2        Burrit

In [13]:
# function to calculate for common venues for each postal code
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['borough'] = toronto_grouped['borough']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Clothing Store,Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Seafood Restaurant,Park
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Brewery,Park,Sandwich Place,Pub,Pizza Place
3,East York,Coffee Shop,Burger Joint,Park,Pizza Place,Pharmacy,Sporting Goods Shop,Bank,Sandwich Place,Furniture / Home Store,Liquor Store
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery


# Create clusters for each Postal Code

In [15]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [16]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('borough', 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=0, n_init=12).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 4, 2, 3, 5, 3, 0])

In [17]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('borough'), on='borough')
toronto_merged.head() 

,postalcode,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store


# Map Clusters

In [18]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [19]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['postalcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Each Cluster

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,The Beaches,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Brewery,Park,Sandwich Place,Pub,Pizza Place
41,M4K,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Brewery,Park,Sandwich Place,Pub,Pizza Place
42,M4L,"The Beaches West, India Bazaar",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Brewery,Park,Sandwich Place,Pub,Pizza Place
43,M4M,Studio District,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Brewery,Park,Sandwich Place,Pub,Pizza Place
44,M4N,Lawrence Park,0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Clothing Store,Restaurant
45,M4P,Davisville North,0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Clothing Store,Restaurant
46,M4R,North Toronto West,0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Clothing Store,Restaurant
47,M4S,Davisville,0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Clothing Store,Restaurant
48,M4T,"Moore Park, Summerhill East",0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Clothing Store,Restaurant
49,M4V,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Clothing Store,Restaurant


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,M6C,Humewood-Cedarvale,1,Fast Food Restaurant,Park,Convenience Store,Grocery Store,Caribbean Restaurant,Field,Bus Line,Sandwich Place,Bar,Restaurant
74,M6E,Caledonia-Fairbanks,1,Fast Food Restaurant,Park,Convenience Store,Grocery Store,Caribbean Restaurant,Field,Bus Line,Sandwich Place,Bar,Restaurant
80,M6M,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",1,Fast Food Restaurant,Park,Convenience Store,Grocery Store,Caribbean Restaurant,Field,Bus Line,Sandwich Place,Bar,Restaurant
81,M6N,"The Junction North, Runnymede",1,Fast Food Restaurant,Park,Convenience Store,Grocery Store,Caribbean Restaurant,Field,Bus Line,Sandwich Place,Bar,Restaurant
98,M9N,Weston,1,Fast Food Restaurant,Park,Convenience Store,Grocery Store,Caribbean Restaurant,Field,Bus Line,Sandwich Place,Bar,Restaurant


In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,M7R,Canada Post Gateway Processing Centre,2,Coffee Shop,Hotel,American Restaurant,Fried Chicken Joint,Sandwich Place,Burrito Place,Donut Shop,Discount Store,Dog Run,Doner Restaurant


In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
1,M1C,"Highland Creek, Rouge Hill, Port Union",3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
2,M1E,"Guildwood, Morningside, West Hill",3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
3,M1G,Woburn,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
4,M1H,Cedarbrae,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
5,M1J,Scarborough Village,3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
7,M1L,"Clairlea, Golden Mile, Oakridge",3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store
9,M1N,"Birch Cliff, Cliffside West",3,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Park,Discount Store,Bakery,Thai Restaurant,Electronics Store


In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,M8V,"Humber Bay Shores, Mimico South, New Toronto",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
89,M8W,"Alderwood, Long Branch",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
90,M8X,"The Kingsway, Montgomery Road, Old Mill North",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
91,M8Y,"Humber Bay, Kings Mill Park, Kingsway Park Sou...",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
92,M8Z,"Kingsway Park South West, Mimico NW, The Queen...",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
93,M9A,Islington Avenue,4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
94,M9B,"Cloverdale, Islington, Martin Grove, Princess ...",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
95,M9C,"Bloordale Gardens, Eringate, Markland Wood, Ol...",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
99,M9P,Westmount,4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery
100,M9R,"Kingsview Village, Martin Grove Gardens, Richv...",4,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Park,Grocery Store,Gym,Fast Food Restaurant,Pool,Bakery


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,M7A,Queens Park,5,Coffee Shop,Park,Burger Joint,Gym,Diner,Café,Seafood Restaurant,Portuguese Restaurant,Mexican Restaurant,Bar


In [31]:
# save as html file
# map_clusters.save('index.html')